In [1]:
%matplotlib nbagg
#%load_ext autoreload

In [2]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from matplotlib import cm
from dask import delayed
from dask import array as da
from copy import deepcopy

import libertem
from libertem import api
from libertem.executor.dask import DelayedJobExecutor
from libertem.executor.inline import InlineJobExecutor

from libertem.udf.base import UDFRunner, UDF

In [3]:
delayed_e = DelayedJobExecutor()
ctx = api.Context(executor=delayed_e)
# inline_ctx = api.Context(executor=InlineJobExecutor())
# dist_ctx = api.Context()

In [4]:
ds = ctx.load(
    "auto",
    path=r"E:\LargeData\LargeData\ER-C-1\groups\data_science\data\reference\EMPAD\BiFeO3\acquisition_12.xml",
)
(scan_y, scan_x, detector_y, detector_x) = ds.shape
mask_shape = np.array((detector_y, detector_x))
cx = detector_x/2
cy = detector_y/2
y = scan_y//2
x = scan_x//2 + 1

In [5]:
def all_ones():
    return np.ones((detector_y, detector_x))

In [6]:
def single_pixel():
    buf = np.zeros((detector_y, detector_x))
    buf[int(cy), int(cx)] = 1
    return buf

Here we use a mask UDF. This UDF uses the LiberTEM computation rather directly and returns a simple numerical result. See further below for examples that use the high-level analysis interface instead.

In [7]:
mask_udf = libertem.udf.masks.ApplyMasksUDF(mask_factories=[all_ones, single_pixel])
sum_udf = libertem.udf.sum.SumUDF()
roi = np.random.choice([True, False], size=ds.shape.nav)

In [16]:
%%time
runner = UDFRunner([mask_udf, sum_udf])
direct_result = runner.run_for_dataset(
    dataset=ds,
    executor=delayed_e,
    roi=roi,
    corrections=None,
    backends=None
)

Wall time: 3.23 s


In [9]:
runner = UDFRunner([mask_udf, sum_udf])
delayed_result = runner.delayed_for_dataset(
    dataset=ds,
    executor=delayed_e,
    roi=roi,
    corrections=None,
    backends=None
)

In [10]:
delayed_result[0]['intensity'].data

dask.array<from-value, shape=(256, 256, 2), dtype=float64, chunksize=(256, 256, 2), chunktype=numpy.ndarray>

In [12]:
delayed_result[0]['intensity'].raw_data

dask.array<concatenate, shape=(65536, 2), dtype=float64, chunksize=(16384, 2), chunktype=numpy.ndarray>

In [11]:
delayed_result[1]['intensity'].data

dask.array<from-value, shape=(128, 128), dtype=float32, chunksize=(128, 128), chunktype=numpy.ndarray>

In [15]:
%%time
delayed()([delayed_result[0]['intensity'].data, delayed_result[1]['intensity'].data]).compute()

Wall time: 3.14 s


[array([[[           nan,            nan],
         [           nan,            nan],
         [2.92068387e+07, 6.49072412e+03],
         ...,
         [           nan,            nan],
         [2.83885469e+07, 6.46167285e+03],
         [           nan,            nan]],
 
        [[2.86121717e+07, 1.14116309e+04],
         [2.89554789e+07, 1.35690986e+04],
         [           nan,            nan],
         ...,
         [           nan,            nan],
         [           nan,            nan],
         [2.83712761e+07, 4.68074365e+03]],
 
        [[           nan,            nan],
         [2.84225564e+07, 1.05717480e+04],
         [2.93321367e+07, 7.03878223e+03],
         ...,
         [2.87344133e+07, 1.21677715e+04],
         [           nan,            nan],
         [           nan,            nan]],
 
        ...,
 
        [[2.61672790e+07, 7.09153076e+03],
         [           nan,            nan],
         [           nan,            nan],
         ...,
         [2.71181

In [17]:
np.allclose(delayed_result[0]['intensity'].data.compute(), direct_result[0]['intensity'].data, equal_nan=True)

True

In [18]:
np.allclose(delayed_result[1]['intensity'].data.compute(), direct_result[1]['intensity'].data)

True